In [1]:
import os
import cv2
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Flatten, Dense, Input, 
                                     Dropout, BatchNormalization,
                                     Conv2D, MaxPooling2D, 
                                     GlobalMaxPooling2D, GlobalAveragePooling2D,)

from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

In [2]:
def cv_show(img, name='image'):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

In [4]:
def padding(img):
    out_w = 128
    out_h = 128
    top_size = int((128 - img.shape[0]) / 2)
    bottom_size = int((128 - img.shape[0]) / 2)
    dif_1 = 128 - (top_size + bottom_size + img.shape[0])
    top_size += dif_1
    left_size = int((128 - img.shape[1]) / 2)
    right_size = int((128 - img.shape[1]) / 2)
    dif_2 = 128 - (left_size + right_size + img.shape[1])
    left_size += dif_2    
    if img.shape[1] <= 128:
        constant = cv2.copyMakeBorder(img, 
                                  top_size, bottom_size, left_size, right_size, 
                                  borderType=cv2.BORDER_CONSTANT, 
                                  value=(255, 255, 255))
    else:
        constant = cv2.copyMakeBorder(img, 
                          top_size, bottom_size, 0, 0, 
                          borderType=cv2.BORDER_CONSTANT, 
                          value=(255, 255, 255))
        constant = cv2.resize(constant, (128, 128))
        
 
    return constant

In [26]:
def preprocessing_img(img):

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(gray, 110, 255, cv2.THRESH_OTSU)   
    gaussian = cv2.GaussianBlur(thresh1, (3, 3), 1)
    kernel = np.zeros((3, 3), np.uint8)
    erosion_1 = cv2.erode(gaussian, kernel, iterations=1)

    img2 = 255 - erosion_1
    img2 = cv2.resize(img2, (64, 64))
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    
    
    return img2

In [27]:
def load_data(path):
    
    fold_list = os.listdir(train_dir)
    trainX_list = []
    trainY_list = []
    testX_list = []
    testY_list = []

    for f in fold_list:
        label = int(f)
        img_list = os.listdir(f'{train_dir}{f}')
        tmp_datasetX = []
        tmp_datasetY = []
        for img_p in img_list:
            try:
                img = cv2.imread(f'{train_dir}{f}/{img_p}')
#                 if (img.shape[0] != 128) or (img.shape[1] != 128):
#                     img = padding(img)
                img = preprocessing_img(img)
#                 print(img.shape)
                label = int(f)
                tmp_datasetX.append(img)
                tmp_datasetY.append(label)
            except Exception as e:
                print(f, img_p)
                print(e)
        
        trainX, testX, trainY, testY = train_test_split(tmp_datasetX, tmp_datasetY, test_size=0.2, random_state=42)
        trainX_list.extend(trainX)
        trainY_list.extend(trainY)
        testX_list.extend(testX)
        testY_list.extend(testY)
    trainX_ary = np.array(trainX_list)
    trainY_ary = np.array(trainY_list)
    testX_ary = np.array(testX_list)
    testY_ary = np.array(testY_list)
    print(f'trainX: {trainX_ary.shape}')
    print(f'trainY: {trainY_ary.shape}')
    print(f'testX: {testX_ary.shape}')
    print(f'testY: {testY_ary.shape}')
        
    return trainX_ary, testX_ary, trainY_ary, testY_ary

In [28]:
train_dir = r'./data-clean/'
df = pd.read_csv('word_idx_df.csv', converters={'idx':str})

In [29]:
trainX, testX, trainY, testY = load_data(train_dir)


trainX: (47930, 64, 64, 3)
trainY: (47930,)
testX: (12391, 64, 64, 3)
testY: (12391,)


In [30]:
trainY = keras.utils.to_categorical(trainY, num_classes=800)
testY = keras.utils.to_categorical(testY, num_classes=800)

In [31]:
print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)

(47930, 64, 64, 3)
(12391, 64, 64, 3)
(47930, 800)
(12391, 800)


In [12]:
print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)

(47930, 128, 128, 3)
(12391, 128, 128, 3)
(47930, 800)
(12391, 800)


In [32]:
import gc
gc.collect()

8463

In [14]:
# trainX_pre = []
# trainY_pre = []
# for x, y in zip(trainX, trainY):
#     tmp_img = preprocessing_img(x)
#     trainX_pre.append(tmp_img)
#     trainY_pre.append(y)

# testX_pre = []
# testY_pre = []
# for x, y in zip(testX, testY):
#     tmp_img = preprocessing_img(x)
#     testX_pre.append(tmp_img)
#     testY_pre.append(y)

# print(trainX.shape)
# trainX = np.vstack((testX, np.array(trainX_pre)))
# print(trainX.shape)
# print(trainY.shape)
# trainY = np.vstack((testY, np.array(trainY_pre)))
# print(trainY.shape)

# print(testX.shape)
# testX = np.vstack((testX, np.array(testX_pre)))
# print(testX.shape)
# print(testY.shape)
# testY = np.vstack((testY, np.array(testY_pre)))
# print(testY.shape)

In [66]:
input_tensor = Input(shape=(64, 64, 3))
model = keras.applications.Xception(include_top=False, 
                                    weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None,
                                    classes=800)
x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
# x = Dense(1024, activation='relu')(x)
# x = BatchNormalization()(x)
predictions = Dense(800, activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)

In [67]:
len(model.layers)

136

In [68]:
for layer in model.layers[:66]:
    layer.trainable = False
for layer in model.layers[66:]:
    layer.trainable = True

In [69]:
model.layers[66].name

'block8_sepconv1_act'

In [70]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 31, 31, 32)   864         input_6[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 31, 31, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 31, 31, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [71]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import optimizers, callbacks

In [72]:
opt = optimizers.Adam(lr=0.001) 
model.compile(loss='categorical_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=0),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')
    ]

In [73]:
model.fit(trainX, trainY, batch_size=32, epochs=50, verbose=2,
          validation_data=(testX, testY), callbacks=callbacks ) 

Epoch 1/50
1498/1498 - 63s - loss: 6.2648 - accuracy: 0.0171 - val_loss: 5.4186 - val_accuracy: 0.0392 - lr: 0.0010
Epoch 2/50
1498/1498 - 63s - loss: 4.2624 - accuracy: 0.1235 - val_loss: 3.8140 - val_accuracy: 0.1763 - lr: 0.0010
Epoch 3/50
1498/1498 - 64s - loss: 3.0933 - accuracy: 0.2834 - val_loss: 3.0437 - val_accuracy: 0.3036 - lr: 0.0010
Epoch 4/50
1498/1498 - 64s - loss: 2.3669 - accuracy: 0.4144 - val_loss: 2.5177 - val_accuracy: 0.3980 - lr: 0.0010
Epoch 5/50
1498/1498 - 64s - loss: 1.8513 - accuracy: 0.5215 - val_loss: 2.3458 - val_accuracy: 0.4513 - lr: 0.0010
Epoch 6/50
1498/1498 - 64s - loss: 1.4758 - accuracy: 0.6051 - val_loss: 2.2139 - val_accuracy: 0.4860 - lr: 0.0010
Epoch 7/50
1498/1498 - 64s - loss: 1.1560 - accuracy: 0.6771 - val_loss: 2.2985 - val_accuracy: 0.4837 - lr: 0.0010
Epoch 8/50
1498/1498 - 64s - loss: 0.8948 - accuracy: 0.7436 - val_loss: 2.1825 - val_accuracy: 0.5297 - lr: 0.0010
Epoch 9/50
1498/1498 - 64s - loss: 0.7004 - accuracy: 0.7921 - val_loss:

In [42]:
model.fit(trainX, trainY, batch_size=32, epochs=50, verbose=2,
          validation_data=(testX, testY), callbacks=callbacks ) 

Epoch 1/50
1498/1498 - 59s - loss: 6.7332 - accuracy: 0.0086 - val_loss: 5.7409 - val_accuracy: 0.0172 - lr: 0.0010
Epoch 2/50
1498/1498 - 58s - loss: 5.2370 - accuracy: 0.0428 - val_loss: 4.6122 - val_accuracy: 0.0803 - lr: 0.0010
Epoch 3/50
1498/1498 - 58s - loss: 4.1020 - accuracy: 0.1320 - val_loss: 4.1929 - val_accuracy: 0.1374 - lr: 0.0010
Epoch 4/50
1498/1498 - 58s - loss: 3.2775 - accuracy: 0.2445 - val_loss: 3.2044 - val_accuracy: 0.2746 - lr: 0.0010
Epoch 5/50
1498/1498 - 58s - loss: 2.6611 - accuracy: 0.3518 - val_loss: 2.8965 - val_accuracy: 0.3331 - lr: 0.0010
Epoch 6/50
1498/1498 - 58s - loss: 2.1966 - accuracy: 0.4414 - val_loss: 2.7709 - val_accuracy: 0.3801 - lr: 0.0010
Epoch 7/50
1498/1498 - 58s - loss: 1.8129 - accuracy: 0.5200 - val_loss: 2.5793 - val_accuracy: 0.4132 - lr: 0.0010
Epoch 8/50
1498/1498 - 59s - loss: 1.4913 - accuracy: 0.5936 - val_loss: 2.5880 - val_accuracy: 0.4237 - lr: 0.0010
Epoch 9/50
1498/1498 - 59s - loss: 1.2218 - accuracy: 0.6540 - val_loss:

In [35]:
model.fit(trainX, trainY, batch_size=32, epochs=50, verbose=2,
          validation_data=(testX, testY), callbacks=callbacks ) 

Epoch 1/50
2996/2996 - 313s - loss: 6.7389 - accuracy: 0.0064 - val_loss: 6.5073 - val_accuracy: 0.0121 - lr: 0.0010
Epoch 2/50
2996/2996 - 316s - loss: 5.3863 - accuracy: 0.0345 - val_loss: 5.0849 - val_accuracy: 0.0510 - lr: 0.0010
Epoch 3/50
2996/2996 - 317s - loss: 4.6345 - accuracy: 0.0815 - val_loss: 4.8776 - val_accuracy: 0.1039 - lr: 0.0010
Epoch 4/50
2996/2996 - 317s - loss: 4.0244 - accuracy: 0.1487 - val_loss: 3.9419 - val_accuracy: 0.1742 - lr: 0.0010
Epoch 5/50
2996/2996 - 319s - loss: 3.4902 - accuracy: 0.2227 - val_loss: 3.7727 - val_accuracy: 0.2083 - lr: 0.0010
Epoch 6/50
2996/2996 - 319s - loss: 3.0054 - accuracy: 0.3009 - val_loss: 3.4232 - val_accuracy: 0.2775 - lr: 0.0010
Epoch 7/50
2996/2996 - 319s - loss: 2.5954 - accuracy: 0.3750 - val_loss: 3.1429 - val_accuracy: 0.3078 - lr: 0.0010
Epoch 8/50
2996/2996 - 319s - loss: 2.2621 - accuracy: 0.4377 - val_loss: 3.1681 - val_accuracy: 0.3542 - lr: 0.0010
Epoch 9/50
2996/2996 - 319s - loss: 1.9781 - accuracy: 0.4936 - 

KeyboardInterrupt: 

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.fit(trainX, trainY, batch_size=32, epochs=10, validation_data=(testX, testY))

In [ ]:
model.fit(trainX, trainY, batch_size=32, epochs=10, validation_data=(testX, testY), workers=-1)

In [ ]:
model.fit(trainX, trainY, batch_size=32, epochs=10, validation_data=(testX, testY))

In [ ]:
model.fit(trainX, trainY, batch_size=32, epochs=50, validation_data=(testX, testY))

In [ ]:
def VGG16(include_top=True,input_tensor=None, input_shape=(128, 128, 1),
          pooling='max',classes=800):
 
    img_input = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
  
    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(1000, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')

    return model

In [ ]:
model = VGG16()

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(trainX, trainY, batch_size=32, epochs=10)